In [1]:
import os

os.chdir("../")

In [2]:
import glob

mscx_path_list = []
for item in glob.glob("./xmls/mscx/*"):
    mscx_path_list.append(os.path.abspath(item))
mscx_path_list.sort()

tstree_path_list = []
for item in glob.glob("./xmls/tstree/*"):
    tstree_path_list.append(os.path.abspath(item))
tstree_path_list.sort()

In [3]:
from alm.lyrics import *
from alm.melody import *
from alm.comparator import *
from alm.utils import io
import pprint

In [4]:
def calc_word_match_rate(mscx_path_list: list, tstree_path_list: list,):
    parser = grammar_parser.GrammarParser("ja_ginza")

    res = []

    if len(mscx_path_list) != len(tstree_path_list):
        return res

    for i in range(len(mscx_path_list)):
        mscx_path = mscx_path_list[i]
        ts_path = tstree_path_list[i]

        lyrics_notes_map = lyrics_extractor.extract_lyrics(mscx_path)
        doc = parser.parse(lyrics_notes_map[lyrics_extractor.LYRICS_KEY])
        lyrics_tree = parser.to_tree(doc)
        words_notes_map = {}
        associating_lyrics_melody.explore_words_in_tree(lyrics_tree, words_notes_map)
        words_list = associating_lyrics_melody.associate_word_list_notes(words_notes_map, lyrics_notes_map)

        melody_tree = time_span_tree.tstree_xml_2_struct(ts_path)

        rate = word_match_rate_calculator.calc_word_match_rate(words_list, melody_tree)
        rate.section_name = io.get_file_name(mscx_path)

        res.append(rate)
    
    return res

In [5]:
word_match_rates = calc_word_match_rate(mscx_path_list, tstree_path_list)

In [9]:
word_match_rate_map = {}
for item in word_match_rates:
    song_name = item.section_name[:-2]
    section = item.section_name[-1]

    if song_name not in word_match_rate_map:
        word_match_rate_map[song_name] = [-1, -1, -1, -1]

    if section == "A":
        word_match_rate_map[song_name][0] = item.total_words_number
        word_match_rate_map[song_name][1] = item.matched_words_number
    elif section == "S":
        word_match_rate_map[song_name][2] = item.total_words_number
        word_match_rate_map[song_name][3] = item.matched_words_number

res = [] 
for key, value in word_match_rate_map.items():
    value.insert(0, key)
    res.append(value)

pprint.pprint(res)

[['BE_FREE', 20, 17, 28, 22],
 ['SUN_SHINE!!!', 45, 27, 21, 11],
 ['beautiful_days', 27, 22, 35, 26],
 ['no_more_war', 59, 36, 17, 10],
 ['たけてん', 28, 23, 38, 26],
 ['オレンジ', 33, 25, 60, 44],
 ['キセキ', 25, 18, 42, 27],
 ['サヨナラから始めよう', 56, 36, 17, 13],
 ['ビリーヴ', 43, 31, 46, 29],
 ['ルーキーズ', 57, 38, 43, 26],
 ['人', 62, 50, 38, 27],
 ['刹那', 24, 18, 25, 19],
 ['君想い', 58, 44, 27, 15],
 ['愛唄', 39, 21, 29, 21],
 ['扉', 45, 32, 29, 25],
 ['旅立ち', 30, 25, 28, 20],
 ['星影のエール', 30, 19, 29, 19],
 ['涙空', 55, 34, 13, 9],
 ['花唄', 42, 30, 29, 24],
 ['遥か', 39, 20, 30, 26]]
